The goal of the notebook is to construct an API which can augment the coordinates by extending the box, such that other methods which are not PBC aware can be used. The indices can be back transformed by creating a hashmap which links every artificial atom outside the bounds of the box to its real counterpart.

In [17]:
from MDAnalysis.lib.distances import distance_array
import numpy as np
from collections import defaultdict

To create the relevant image points, the slick algorithm used in the implementation of periodic KDtree is used. Another hashmap is also contructed which stores the information of multiple images and their linkage to the original coordinates.

In [10]:
def get_coords(box,Npoints):
    return (np.random.uniform(low=0,high=1.0,size=(Npoints,3))*box[:3]).astype(np.float32)

In [19]:
def augment(points, cutoff, box):
    """
    Augments the cordinates and returns the coordinates in 
    the extended box, such that all the atoms in the box 
    are PBC aware
    
    Parameters
    ----------
    points : coordinates of dimension (N, 3)
    cutoff : cutoff radius 
    box : PBC aware box
    
    Returns
    -------
    augmented_coords : similar to points but added coordinates 
    mapid : hash map with the original to artificial mapping
    """
    a, b, c = box[:3]
    dm = np.array([[a, 0, 0], [0, b, 0], [0, 0, c]], dtype=np.float32)
    rm = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype=np.float32)
    
    augmented_coords = points.copy() ## So as to not change the original coordinates
    mapid = defaultdict(list) # hash maps
    for idx, coords in enumerate(points):
        images = list()
        distances = np.dot(rm, coords)
        displacements = list(dm[np.where(distances < cutoff)[0]])
        distances = np.einsum('ij,ij->i', rm,dm - coords)
        displacements.extend(list(-dm[np.where(distances < cutoff)[0]]))
        n_displacements = len(displacements)
        if n_displacements > 1:
            for start in range(n_displacements - 1, -1, -1):
                for i in range(start+1, len(displacements)):
                    displacements.append(displacements[start]+displacements[i])
        images.extend([coords + d for d in displacements])
        updated_N = len(augmented_coords)
        if len(images) > 0 :
            augmented_coords = np.append(augmented_coords, [images[i] for i in range(len(images))], axis = 0)
            for newidx in range(len(images)):
                mapid[updated_N + newidx].append(idx)
    return augmented_coords, mapid

In [63]:
def inverse_augment(pairs, mapid, N):
    """
    Returns the set of unique pairs
    
    pairs is a list of pairs which needs to be replaced
    
    Parameters
    ----------
    pairs : tuple of indices
    mapid : hash map returned from augment 
    N : Number of points in original indices
    
    Returns
    -------
    numpy array of unique bonds
    """
    pairs = np.asarray(pairs)
    for idx, (a,b) in enumerate(pairs):
        for ind, val in enumerate([a,b]):
            if val >= N:
                pairs[idx, ind] = mapid[val][0]
        pairs[idx,:] = np.sort(pairs[idx, :])
    return np.unique(pairs, axis = 0) 

# Check with and without the augment

Lets test the implementation against brute force for the evaluation of pairs which are within a certain cutoff distance.

In [34]:
def bf_select(points, maxdist, box = None):
    bonds = []
    for i,coordinate in enumerate(points):
        dist = distance_array(points[i][None, :], points[i + 1:],box=box)[0]
        idx = np.where((dist <= maxdist))[0]
        for a in idx:
            j = i + 1 + a
            atom_j = points[j]
            bonds.append((i, j))
    return np.asarray(bonds)

In [67]:
def aug_select(points, maxdist, box):
    augment_coords, mapid = augment(points, maxdist, box)
    art_bonds = bf_select(augment_coords, maxdist)
    bonds = inverse_augment(art_bonds, mapid, len(points))
    return bonds

In [78]:
# Initialize
box = np.array([100,100,100,90,90,90],dtype=np.float32)
points = get_coords(box,Npoints=10000)
maxdist = 4.0

In [79]:
bonds = bf_select(points, maxdist, box)
len(bonds)

13407

In [80]:
bonds_aug = aug_select(points, maxdist, box)
len(bonds_aug)

13407

In [81]:
%timeit bf_select(points, maxdist, box)

2.48 s ± 78.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
%timeit aug_select(points, maxdist, box)

1.47 s ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
